In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import schedule_Generator as sg
import datetime
import time
import re

## 2021-09-07 Example, Tram 25 ULB

In [2]:
Vehicle_Position = pd.read_csv("../data/Combined_CSV.csv")
Vehicle_Position.head()

,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-06 07:54:46.924000,1,8161,1,8012
1,2021-09-06 07:54:46.924000,1,8162,0,8142
2,2021-09-06 07:54:46.924000,1,8162,0,8282
3,2021-09-06 07:54:46.924000,1,8731,0,8111
4,2021-09-06 07:54:46.924000,1,8162,1,8062


In [3]:
Vehicle_Position_sub = Vehicle_Position[Vehicle_Position["Time"].apply(lambda x: x[0:10]) == "2021-09-07"].reset_index(drop=True)
print(Vehicle_Position_sub.shape)
Vehicle_Position_sub.head()

(1423791, 5)


,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-07 00:00:20.209000,20,2081,60,1096
1,2021-09-07 00:00:20.209000,20,1900,65,1096
2,2021-09-07 00:00:20.209000,29,1528,0,2594
3,2021-09-07 00:00:20.209000,34,9978,173,1824
4,2021-09-07 00:00:20.209000,34,6432,87,1278


In [4]:
trips = pd.read_csv('../data/gtfs3Sept/trips.txt')
trips_line = trips.loc[trips['route_id']==25,:]
trip_line_head = trips_line.loc[trips_line['trip_headsign']=='BOONDAEL GARE',:].reset_index(drop=True)
print(trip_line_head.shape)
trip_line_head.head()

(2107, 7)


,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,25,236896051,113149920236896051,BOONDAEL GARE,1,8925469,025t0256
1,25,236896051,113149922236896051,BOONDAEL GARE,1,8925474,025t0256
2,25,236896051,113149925236896051,BOONDAEL GARE,1,8925469,025t0256
3,25,236896051,113149929236896051,BOONDAEL GARE,1,8925473,025t0256
4,25,236896051,113149932236896051,BOONDAEL GARE,1,8925474,025t0256


In [5]:
calendar = pd.read_csv('../data/gtfs3Sept/calendar.txt')
calendar_week = calendar.loc[calendar['start_date']==20210906,:]
calendar_week_day = calendar_week.loc[calendar_week['tuesday']==1,:].reset_index(drop=True)
print(calendar_week_day.shape)
calendar_week_day.head()

(10, 10)


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,237645001,1,1,1,1,1,0,0,20210906,20210917
1,237648001,1,1,1,1,1,0,0,20210906,20210917
2,238202001,1,1,1,1,1,0,0,20210906,20210917
3,238119001,1,1,1,1,1,0,0,20210906,20210917
4,238201001,1,1,1,1,1,0,0,20210906,20210917


In [6]:
trip_line_date_head = pd.merge(left=trip_line_head, right=calendar_week_day, on='service_id', how='inner').loc[:,['route_id','service_id','trip_id']]
print(trip_line_date_head.shape)
trip_line_date_head.head()

(288, 3)


,route_id,service_id,trip_id
0,25,238202001,113480343238202001
1,25,238202001,113480346238202001
2,25,238202001,113480348238202001
3,25,238202001,113480356238202001
4,25,238202001,113480357238202001


In [7]:
stop_times = pd.read_csv('../data/gtfs3Sept/stop_times.txt')
stop_times

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,112387248235954071,21:07:00,21:07:00,4014,1,0,0
1,112387248235954071,21:09:00,21:09:00,3231,2,0,0
2,112387248235954071,21:10:08,21:10:08,3232,3,0,0
3,112387248235954071,21:11:00,21:11:00,3233,4,0,0
4,112387248235954071,21:11:43,21:11:43,3239,5,0,0
...,...,...,...,...,...,...,...
2820504,113028649236519600,07:29:00,07:29:00,6427F,17,0,0
2820505,113028649236519600,07:30:00,07:30:00,6430F,18,0,0
2820506,113028649236519600,07:31:35,07:31:35,5066F,19,0,0
2820507,113028649236519600,07:33:00,07:33:00,5068F,20,0,0


## Schedule Waiting Time

In [8]:
merged_stop_times = pd.merge(left = trip_line_date_head,right=stop_times,
         on="trip_id",how="inner")[["route_id",
                                    "service_id",
                                    "trip_id",
                                    "arrival_time",
                                    "departure_time",
                                    "stop_id",
                                    "stop_sequence"]].sort_values(by = ["arrival_time"],
                                                                  ascending=True).reset_index(drop=True)
print(merged_stop_times.shape)
merged_stop_times.head(10)

(3809, 7)


,route_id,service_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,25,238202001,113480365238202001,04:46:00,04:46:00,2720F,1
1,25,238202001,113480365238202001,04:47:57,04:47:57,6201F,2
2,25,238202001,113480365238202001,04:49:00,04:49:00,6202,3
3,25,238202001,113480365238202001,04:50:00,04:50:00,2794F,4
4,25,238202001,113480343238202001,05:04:00,05:04:00,0539,1
5,25,238202001,113480343238202001,05:05:00,05:05:00,0536,2
6,25,238202001,113480356238202001,05:07:00,05:07:00,2397F,1
7,25,238202001,113480356238202001,05:08:00,05:08:00,5407F,2
8,25,238202001,113480343238202001,05:08:00,05:08:00,5762,3
9,25,238202001,113480343238202001,05:09:00,05:09:00,2720F,4


In [9]:
stops = pd.read_csv('../data/gtfs3Sept/stops.txt')[["stop_id","stop_name","stop_lat","stop_lon"]]
print(stops.shape)
stops.head()

(2998, 4)


,stop_id,stop_name,stop_lat,stop_lon
0,0089,MONTGOMERY,50.838006,4.408970
1,0470F,SIMONIS,50.863666,4.329612
2,0471,SIMONIS,50.863732,4.329236
3,0472,SIMONIS,50.863543,4.329023
4,0473F,SIMONIS,50.863418,4.330031


In [10]:
stops[stops["stop_name"] == "ULB"]

,stop_id,stop_name,stop_lat,stop_lon
1535,3513,ULB,50.813926,4.384942
1559,3559,ULB,50.813899,4.384459
2021,5407,ULB,50.814061,4.384119
2022,5407F,ULB,50.814061,4.384119
2054,5462F,ULB,50.813657,4.384332


In [11]:
merge_ULB = pd.merge(left=merged_stop_times,
         right=stops[stops["stop_name"] == "ULB"],
         on = "stop_id",
         how = "inner")
merge_ULB

,route_id,service_id,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_name,stop_lat,stop_lon
0,25,238202001,113480356238202001,05:08:00,05:08:00,5407F,2,ULB,50.814061,4.384119
1,25,238202001,113480381238202001,05:26:00,05:26:00,5407F,2,ULB,50.814061,4.384119
2,25,238202001,113481493238202001,05:44:00,05:44:00,5407F,2,ULB,50.814061,4.384119
3,25,238202001,113480361238202001,05:47:00,05:47:00,5407F,16,ULB,50.814061,4.384119
4,25,238202001,113481495238202001,06:00:00,06:00:00,5407F,2,ULB,50.814061,4.384119
5,25,238202001,113480369238202001,06:03:00,06:03:00,5407F,16,ULB,50.814061,4.384119
6,25,238202001,113481500238202001,06:17:00,06:17:00,5407F,2,ULB,50.814061,4.384119
7,25,238202001,113480360238202001,06:22:00,06:22:00,5407F,16,ULB,50.814061,4.384119
8,25,238202001,113480582238202001,06:30:00,06:30:00,5407F,16,ULB,50.814061,4.384119
9,25,238202001,113480583238202001,06:38:00,06:38:00,5407F,16,ULB,50.814061,4.384119


In [12]:
ULB_schedule_diff = np.diff(merge_ULB["arrival_time"].apply(lambda x: int(x[0:2])*60 + int(x[3:5])))
ULB_schedule_diff

array([18, 18,  3, 13,  3, 14,  5,  8,  8,  6,  3,  9,  8,  6,  2,  8,  6,
        6,  7,  6,  8,  7,  6,  7,  6,  6,  6,  6,  7,  7,  7,  7,  6, 11,
        7,  5,  5,  6,  6,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  9,  8,  8,  8,  8,  8,  9,  8,  7,  8,  5,  2,  7,
        5,  5,  6,  8,  7,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  5,  6,  7,  7,  7,  9,
        9, 11, 10,  9, 10,  9, 12, 10, 13, 12, 11, 12, 15, 16, 16, 15, 15,
       15, 15, 15, 15, 15, 15, 15, 15, 20], dtype=int64)

In [13]:
print("ULB Schedule Wating Time: {} mins ".format(np.sum(ULB_schedule_diff**2)/(2*np.sum(ULB_schedule_diff))))

ULB Schedule Wating Time: 4.766778523489933 mins 


## Actual Waiting Time

In [14]:
Vehicle_Position_sub["DirectionID"] = Vehicle_Position_sub["DirectionID"].astype("str")

In [15]:
Vehicle_Position_sub.head()

,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-07 00:00:20.209000,20,2081,60,1096
1,2021-09-07 00:00:20.209000,20,1900,65,1096
2,2021-09-07 00:00:20.209000,29,1528,0,2594
3,2021-09-07 00:00:20.209000,34,9978,173,1824
4,2021-09-07 00:00:20.209000,34,6432,87,1278


In [17]:
Vehicle_Position_sub.dtypes

Time                 object
LineID                int64
DirectionID          object
DistanceFromPoint     int64
PointID               int64
dtype: object

In [27]:
ULB_Vehicle = Vehicle_Position_sub.loc[Vehicle_Position_sub["PointID"]==5407].reset_index(drop=True)
ULB_Vehicle

,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-07 03:07:56.444000,25,6487,42,5407
1,2021-09-07 03:08:28.599000,25,6487,42,5407
2,2021-09-07 03:08:58.706000,25,6487,42,5407
3,2021-09-07 03:09:30.758000,8,5229,280,5407
4,2021-09-07 03:09:30.758000,25,6487,42,5407
5,2021-09-07 03:45:09.748000,25,6487,0,5407
6,2021-09-07 03:45:40.059000,25,6487,142,5407
7,2021-09-07 03:46:41.091000,25,6487,42,5407
8,2021-09-07 03:47:11.504000,25,6487,77,5407
9,2021-09-07 03:47:42.431000,25,6487,77,5407


In [28]:
ULB_Vehicle_25 = ULB_Vehicle.loc[ULB_Vehicle["LineID"] == 25]
ULB_Vehicle_25

,Time,LineID,DirectionID,DistanceFromPoint,PointID
0,2021-09-07 03:07:56.444000,25,6487,42,5407
1,2021-09-07 03:08:28.599000,25,6487,42,5407
2,2021-09-07 03:08:58.706000,25,6487,42,5407
4,2021-09-07 03:09:30.758000,25,6487,42,5407
5,2021-09-07 03:45:09.748000,25,6487,0,5407
6,2021-09-07 03:45:40.059000,25,6487,142,5407
7,2021-09-07 03:46:41.091000,25,6487,42,5407
8,2021-09-07 03:47:11.504000,25,6487,77,5407
9,2021-09-07 03:47:42.431000,25,6487,77,5407
10,2021-09-07 03:48:12.741000,25,6487,77,5407


In [50]:
list(stops["stop_id"].unique())

['0089',
 '0470F',
 '0471',
 '0472',
 '0473F',
 '0501',
 '0506',
 '0511',
 '0516',
 '0521',
 '0526',
 '0529',
 '0531',
 '0520161',
 '0536',
 '0539',
 '0601',
 '0600162',
 '0600262',
 '0600362',
 '0600462',
 '0600562',
 '0600662',
 '0600762',
 '0600862',
 '0600962',
 '0601062',
 '0601162',
 '0601262',
 '0606',
 '0611',
 '0600263',
 '0610163',
 '0610363',
 '0610463',
 '0610563',
 '0610663',
 '0616',
 '0610263',
 '0621',
 '0620164',
 '0626',
 '0631',
 '0636',
 '0702',
 '0703',
 '0704',
 '0650165',
 '0650265',
 '0705',
 '0660166',
 '0660266',
 '0706',
 '0670167',
 '0670267',
 '0670367',
 '0722',
 '0723',
 '0724',
 '0725',
 '0726',
 '0801',
 '0806',
 '0811',
 '0810169',
 '0810269',
 '0810369',
 '0810469',
 '0816',
 '0821',
 '0820168',
 '0820268',
 '0826',
 '0901',
 '0900170',
 '0900270',
 '0900370',
 '0900470',
 '0900570',
 '0906',
 '1008',
 '1009',
 '1010',
 '1014F',
 '1015',
 '1016',
 '1018',
 '1019',
 '1024',
 '1030',
 '1042',
 '1043',
 '1046G',
 '1047F',
 '1048F',
 '1049F',
 '1051',
 '1

In [51]:
list(Vehicle_Position["PointID"].unique())

[8012,
 8142,
 8282,
 8111,
 8062,
 8281,
 8102,
 8162,
 8151,
 8081,
 8733,
 8741,
 8041,
 8401,
 8332,
 8441,
 8743,
 8442,
 8754,
 8341,
 8302,
 8472,
 616,
 506,
 601,
 6253,
 702,
 5701,
 6178,
 6179,
 2338,
 5770,
 6209,
 1985,
 5917,
 726,
 4501,
 516,
 1984,
 704,
 6423,
 5953,
 8211,
 8011,
 8262,
 8671,
 8202,
 8641,
 8261,
 8711,
 8272,
 8662,
 8242,
 8702,
 8251,
 8061,
 8042,
 8742,
 8361,
 8321,
 8422,
 8462,
 8382,
 8763,
 8362,
 8421,
 8824,
 8834,
 8803,
 8794,
 901,
 9969,
 5706,
 5308,
 5352,
 5758,
 6457,
 5772,
 9963,
 9056,
 5222,
 2635,
 5355,
 6421,
 5271,
 5466,
 5269,
 5416,
 5258,
 5282,
 5409,
 5230,
 5404,
 5470,
 2014,
 6314,
 5228,
 5422,
 5462,
 1674,
 1684,
 1681,
 470,
 9600,
 1780,
 1270,
 2250,
 5048,
 5044,
 1334,
 4111,
 1732,
 1180,
 1747,
 1782,
 1083,
 1755,
 4268,
 1768,
 5076,
 4228,
 4347,
 4297,
 1455,
 6866,
 471,
 6800,
 1307,
 5001,
 5103,
 2636,
 6865,
 2805,
 472,
 1016,
 5168,
 5105,
 1030,
 4213,
 1792,
 1197,
 4500,
 1262,
 4555,
 32

In [45]:
re.sub("[A-Za-z]","",string = "04FF7F3F")

'0473'

In [58]:
a = list(Vehicle_Position["PointID"].apply(lambda x: str(x)).unique())

In [59]:
b = list(stops["stop_id"].apply(lambda x: re.sub("[A-Za-z]","",string=x)))

In [64]:
print(len(b),len(set(b)))

2998 2842


In [65]:
sum1 = 0
for item in a:
    if item in b:
        sum1 +=1
print(sum1)

2246


In [63]:
len(a)

2367

In [35]:
merge_Vehicle = Vehicle_Position_sub.merge(stops,
        left_on="DirectionID",
        right_on="stop_id",
        how = "inner")

In [40]:
merge_Vehicle.sort_values(by="stop_name")["stop_name"].unique()

array(['ADEPS', 'ALBERT', 'ALBERT I', 'AMBIORIX', 'ANNEESSENS',
       'ARCHIDUCS', 'ARSENAL', 'AUDERGHEM-SHOP.', 'AVENUE DU ROI',
       'BASCULE', 'BEAULIEU', 'BEEKKANT', 'BELGICA', 'BERVOETS', 'BIZET',
       'BLYCKAERTS', 'BOCKSTAEL', 'BORDET STATION', 'BOTANIQUE',
       'BOURDON', 'BOURSE', 'BRABANCONNE', 'BUDA', 'BUYL', 'CARENE',
       'CERIA', "CIM. D'IXELLES", 'CIM. DE BRUXELLES', 'CIM. DE JETTE',
       'CLEMENCE EVERARD', 'CLEMENCEAU', 'CONDOR', 'DA VINCI', 'DAILLY',
       'DANSAERT', 'DE BROUCKERE', 'DECROLY', 'DELACROIX', 'DELTA',
       'DEMEY', 'DIAMANT', 'DIEWEG', 'DIGUE DU CANAL', 'ELBERS',
       'ELISABETH', 'ERASME', 'ESPLANADE', 'ETANGS NOIRS', 'FLAGEY',
       'FOREST (BERVOETS)', 'FOREST NATIONAL', 'GARE CENTRALE',
       'GARE DE BERCHEM', "GARE DE L'OUEST", 'GARE DE LINKEBEEK',
       'GARE DU MIDI', 'GARE DU NORD', 'GEORGES HENRI', 'GLIBERT',
       'GRAND SABLON', 'GRAND-PLACE', 'GUILL. DE GREEF', 'GULLEDELLE',
       'HAREN', 'HEEMBEEK', 'HEILIGENBORRE', '